# Import Library

In [7]:
pip install -U treelib

  Using cached treelib-1.6.1.tar.gz (24 kB)
  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18385 sha256=2e354407b8a2392cac401b1914fb665586f42d1ea633d4bcab93725f260456f2
  Stored in directory: /Users/tomdenice/Library/Caches/pip/wheels/43/de/32/fd270cc5f06de2bebcdaa666e9350b95f54d983212a7aba9ad
Successfully built treelib
Note: you may need to restart the kernel to use updated packages.


In [8]:
from math import sqrt, exp
from math import factorial as fact
import numpy as np
from scipy.stats import norm
from treelib import Tree, Node

# **General Option Class**

In [9]:
class Option:

    def __init__(self, spot: float, strike: float, maturity: float, risk_free_rate: float, dividend: float,
                 volatility: float, option_type: str, side: str):
        self.spot = spot
        self.strike = strike
        self.mat = maturity
        self.rate = risk_free_rate
        self.div = dividend
        self.vol = volatility
        self.type = option_type
        self.side = side

# Option Pricing with Binomial Model

In [10]:
class Binomial_Option_Pricing(Option):

    def __init__(self, spot, strike, maturity, rate, vol, option_type, side, period:int, dividend: float = None):
        Option.__init__(self, spot, strike, maturity, rate, dividend, vol, option_type, side)
        self.period = period
        self.dt = maturity/period
        self.u = np.exp(vol * np.sqrt(self.dt))
        self.d = np.exp(-vol * np.sqrt(self.dt))
        self.p = (np.exp(rate * self.dt) - self.d) / (self.u - self.d)

    @staticmethod
    def binom_coeff(n, k):
        return fact(n) / (fact(n-k) * fact(k))

    def stock_values(self):
        return [self.spot * (self.u) ** i * (self.d) ** (self.period-i) for i in range(self.period + 1)]

    def proba_issues(self):
        return [self.binom_coeff(self.period, i) * (self.p) ** i * (1-self.p) ** (self.period - i) for i in range(self.period + 1)]

    def payoff(self):
        return [max(stock_val - self.strike, 0) if self.type == "call" else max(self.strike - stock_val, 0) for
                stock_val in self.stock_values()]

    def pricing(self):
        price = 0
        for i, j in zip(self.payoff(), self.proba_issues()):
            price += i*j
        return exp(-self.rate * self.mat) * price

    def plot_tree(self):
        tree = Tree()
        tree.create_node(f"Spot Price : {self.spot}", "parent")
        for i, j, k in zip(self.stock_values(), self.proba_issues(), self.payoff()):
            tree.create_node(f"S_t={round(i, 2)} - Payoff={round(k, 2)} - "
                             f"Proba={round(j, 2)}", f"{i}", parent="parent")

        tree.show()

# Tree Plot with Binomial Model

In [11]:
derivatives = Binomial_Option_Pricing(50, 40, 1, 0.05, 0.2, "call", "long", 8, 0)
derivatives.plot_tree()

Spot Price : 50
├── S_t=28.4 - Payoff=0 - Proba=0.0
├── S_t=32.71 - Payoff=0 - Proba=0.02
├── S_t=37.68 - Payoff=0 - Proba=0.09
├── S_t=43.41 - Payoff=3.41 - Proba=0.19
├── S_t=50.0 - Payoff=10.0 - Proba=0.27
├── S_t=57.6 - Payoff=17.6 - Proba=0.24
├── S_t=66.34 - Payoff=26.34 - Proba=0.13
├── S_t=76.42 - Payoff=36.42 - Proba=0.04
└── S_t=88.03 - Payoff=48.03 - Proba=0.01



In [15]:
print(f"Option price estimated with binomial model is {round(derivatives.pricing(), 2)}$")

Option price estimated with binomial model is 12.33$
